<h3><center>Reconhecedor de Dígitos numéricos</center></h3>
<h4>Rede Neural Simples</h4>
<li>Projeto de estudo de rede neurais multicamadas (duas).</li>
<li>O desafio apresentado está no link do kaggle, que é uma base de dados de arquivos abertos para estudo de Data Science e pesquisas estatisticas</li>
<li>A proposta do desafio consiste em implementar uma rede neural que seja capaz de identificar digitos escritos a mão em imagens de resulução de 28X28 pixel que são baixados num formato de array com o labe, valores em tons de cinza que variam de 0 (branco) a 1 (preto) compondo a estrutura da matriz que forma a imagem.
<br/><br/>
<center><img src="utils/images/grey_scale_matrix.png"></center>



<h4>Import de Bibliotecas utilizadas<h4>

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

<h4>Importação da base de dados dos dados</h4>
<li>Fonte dos dados de estudo: <pre>https://www.kaggle.com/c/digit-recognizer/data?select=train.csv </pre>

In [2]:
treino = pd.read_csv('utils/data/train.csv')
teste = pd.read_csv('utils/data/test.csv')

<h4>Tratamento dos dados</h4>

In [4]:
# Convertendo os dados de treino e teste para arrays NumPy
# Isso permite manipular os dados de forma mais eficiente com as operações NumPy
treino = np.array(treino)
teste = np.array(teste)

# Embaralha as amostras de treino para garantir que a rede neural aprenda de uma maneira 
# mais generalizada e não memoriza a sequência de dados
np.random.shuffle(treino)


# Transpõe as matrizes de dados de treino e teste para facilitar a separação de 
# características (X) e rótulos (Y)
dados_teste = teste.T
dados_treino = treino.T

# A primeira linha dos dados de treino corresponde aos rótulos (Y), que representam 
# os valores de saída esperados
Y_treino = dados_treino[0]

# Todas as demais linhas dos dados de teste e treino correspondem às características (X),
# ou seja, os valores de entrada que serão usados como base para o modelo prever o Y correspondente.
X_teste = dados_teste[:]
X_treino = dados_treino[1:]


# Normalização dos dados
# Divide cada valor por 255 para normalizar os valores dos pixels das imagens entre 0 e 1.
# Isso ajuda a reduzir o impacto de diferenças na escala dos valores, o que melhora a performance
# do modelo de rede neural durante o treinamento e a previsão.
X_teste = X_teste / 255.
X_treino = X_treino / 255.

# Armazena o número de colunas (amostras) para treino e teste, respectivamente
_, c_treino = X_treino.shape
_, c_teste = X_teste.shape

<h4>Visualização de Imagens do Dataset</h4>
<li>Função para mostrar imagens de amostras aleatórias do dataset</li>

In [5]:
def mostrar_imagens(indice):
    imagem = X_teste[:, indice, None]
    imagem = imagem.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(imagem, interpolation='nearest')
    plt.show()

<h4>Inicialização da Rede Neural</h4>
<li>Inicia os pesos e vieses aleatoriamente para a camada de entrada e a camada oculta</li>

In [6]:
def montar_rede():
    W1 = np.random.rand(10, 784) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, W2, b1, b2

<h4>Funções de Ativação</h4>
<li>Inicia os pesos e vieses aleatoriamente para a camada de entrada e a camada oculta</li>
<li>A função de ativação ajuda a introduzir não-linearidade no modelo, permitindo que ele aprenda padrões complexos nos dados.</h4>

In [ ]:
# ReLU (Rectified Linear Unit)
def ReLU(Z):
    # A função ReLU retorna o valor de entrada (Z) se for positivo,
    # ou zero se for negativo. Essa função é muito usada em redes neurais,
    # especialmente em camadas intermediárias, pois ajuda a resolver problemas
    # como o desaparecimento do gradiente durante o treinamento.
    # Em termos práticos, ela "ativa" apenas os neurônios que contribuem
    # positivamente para a saída, mantendo zero para valores negativos.
    return np.maximum(0, Z)

# Softmax
def softmax(Z):
    # A função softmax é usada na camada de saída da rede neural para converter
    # os valores de saída (logits) em uma distribuição de probabilidade.
    # Cada valor de entrada é transformado para estar entre 0 e 1, e a soma
    # de todos os valores da saída será igual a 1, formando uma distribuição
    # de probabilidade entre as classes.
    
    # Calcula a exponencial de cada valor de entrada, criando a base para a
    # distribuição de probabilidades. Isso ajuda a destacar valores maiores,
    # aumentando sua probabilidade relativa.
    e = np.exp(Z)
    
    # Divide cada valor pela soma de todas as exponenciais, garantindo que
    # a soma dos valores da saída seja 1 (uma probabilidade).
    return e / e.sum(axis=0)

<h4>Propagação Feedforward</h4>
<li>Propagação da entrada até a saída, retornando os valores calculados nas camada</li>